In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!nvidia-smi

## T5 fine-tuning

This notebook is to showcase how to fine-tune [T5 model](https://arxiv.org/abs/1910.10683) with Huggigface's [Transformers](https://github.com/huggingface/transformers/) to solve different NLP tasks using text-2-text approach proposed in the T5 paper. For demo I chose 3 non text-2-text problems just to reiterate the fact from the paper that how widely applicable this text-2-text framework is and how it can be used for different tasks without changing the model at all.



In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

## Model

We'll be using the awesome [pytorch-lightning](https://github.com/PytorchLightning/pytorch-lightning) library for training. Most of the below code is adapted from here https://github.com/huggingface/transformers/blob/master/examples/lightning_base.py

The trainer is generic and can be used for any text-2-text task. You'll just need to change the dataset. Rest of the code will stay unchanged for all the tasks.

This is the most intresting and powrfull thing about the text-2-text format. You can fine-tune the model on variety of NLP tasks by just formulating the problem in text-2-text setting. No need to change hyperparameters, learning rate, optimizer or loss function. Just plug in your dataset and you are ready to go!

In [ ]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams

    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path, cache_dir="/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/models_cache")
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, cache_dir="/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/models_cache")

  def is_logger(self):
    return self.trainer.proc_rank <= 0

  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        lm_labels=lm_labels,
    )

  def _step(self, batch):
    lm_labels = batch["target_ids"]
    lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        lm_labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}

  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}

  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]

  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
    if self.trainer.use_tpu:
      xm.optimizer_step(optimizer)
    else:
      optimizer.step()
    optimizer.zero_grad()
    self.lr_scheduler.step()

  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

Let's define the hyperparameters and other arguments. You can overide this `dict` for specific task as needed. While in most of cases you'll only need to change the `data_dir`and `output_dir`.

Here the batch size is 8 and gradient_accumulation_steps are 16 so the effective batch size is 128

In [ ]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/ft_T5/fineTunedT5s", # path to save the checkpoints
    model_name_or_path='t5-large',
    tokenizer_name_or_path='t5-large',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

## Generation (or Loading of Dataset)

### Format of the Data :

The dataset should be carefully curated and annotated to support two key objectives: first, to classify the type of document referenced in a given query, and second, to generate an improved, reformulated version of the query. 

Each sample in the dataset must consist of a raw query, a corresponding document type label, and a high-quality, rewritten query that maintains the original intent while enhancing clarity and precision. 

The document types include labels such as PUA, Promesse, and Cession, as well as more nuanced categories like Protocole (PROTO), Bails (either general or with specific details), Garantie (Gap), Acte réitératif (réitératif), Pacte (d'associé), and Transaction (transactionnel, including related forms like protocole transactionnel or transac).

Below is a detailed list of the document categories along with example lowercase regex patterns for each. 
These patterns are designed to match the specific keywords in a case-insensitive context (assuming the text is converted to lowercase before matching):

- PUA
Regex: \bpua\b

- Promesse
Regex: \bpromesse\b

- Cession
Regex: \bcession\b

- Protocole (proto)
Regex: \bprotocole\b|\bproto\b

- Bails
Regex: \bbail(s)?\b => (This pattern matches both the singular "bail" and the plural "bails".)

- Garantie (gap)
Regex: \bgarantie\b|\bgap\b

- Acte réitératif (réitératif)
Regex: \bacte réitératif\b|\bréitératif\b

- Pacte
Regex: \bpacte\b

- Transaction / transactionnel
Regex: \btransaction(?:nel)?\b|\btransac\b =>(This pattern covers both "transaction" and its adjectival form "transactionnel", as well as a shorthand "transac".)

Below is an example of how each data sample in your dataset might be structured. In this JSON-like format, the input field contains the original query text, while the output field is an object that holds both the classified document type and the improved, reformulated query:

{
  "input": "I need the contract details for the promise regarding the asset transfer",
  "output": {
    "type": "promesse",
    "reformulated_query": "Provide the contractual details for the asset transfer promise"
  }
}

#### Load Dataset and prepare training and Validation sets 

### Prepare Dataset

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base', cache_dir="/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/models_cache")

The dataset below takes care of reading the review files and processing the examples in text-2-text format.

It cleans the review text by removing the html tags. It also appends the eos token `</s>` at the end of input and target as required by the T5 model

For T5 max input length is 512 and we can choose the max length for target sequence depending upon our dataset. The `T5Tokenizer` encodes both 'postive' and 'negative' as a single ids so I chose the max target length 2, extra 1 for the `</s>` token

In [ ]:
class QueryDataset(Dataset):
    def __init__(self, tokenizer, csv_file, max_len=512, target_max_len=128):
        """
        Initializes the dataset.
        
        Args:
          tokenizer: Tokenizer to convert text to input IDs.
          csv_file: Path to the CSV file containing the data.
          max_len: Maximum length for tokenizing the input query.
          target_max_len: Maximum length for tokenizing the target output.
        """
        self.tokenizer = tokenizer
        self.data = pd.read_csv(csv_file)
        self.max_len = max_len
        self.target_max_len = target_max_len
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()
        src_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {
            "source_ids": source_ids,
            "source_mask": src_mask,
            "target_ids": target_ids,
            "target_mask": target_mask
        }

    def _build(self):
        for _, row in self.data.iterrows():
            # Extract the initial query and target components.
            initial_query = row['initial_query'].strip()
            doc_type = row['type'].strip()
            new_query = row['new_query'].strip()
            
            # Construct the target string.
            # Example format: "promesse : Provide the contractual details..."
            target = f"type: {doc_type} ; new_query: {new_query} </s>"
            
            # Append a termination token to the input query.
            input_text = initial_query + " </s>"
            
            # Tokenize input text.
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_text],
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors="pt"
            )
            
            # Tokenize target text.
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target],
                max_length=self.target_max_len,
                padding='max_length',
                truncation=True,
                return_tensors="pt"
            )
            
            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
# Create Dataset
path_to_csv = ""
dataset = QueryDataset(tokenizer, path_to_csv, max_len=512, target_max_len=128)
len(dataset)

In [ ]:
data = dataset[28]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

### Train

In [ ]:
args_dict.update({' path_to_csv': 'TO UPDATE', 'output_dir': '/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/ft_T5/fineTunedT5s', 'num_train_epochs':2})
args = argparse.Namespace(**args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

Define the `get_dataset` function to return the dataset. The model calls this function to get the train and val datasets. We are defining a dataset function so that we won't need to modify the model code at all. Redefine the function to return different dataset according to the problem. While this is not the best solution for now this works

In [ ]:
def get_dataset(tokenizer, type_path, args):
  return QueryDataset(tokenizer=tokenizer, path_to_csv=args.path_to_csv, max_len=args.max_seq_length)

**Initialize model**

In [ ]:
model = T5FineTuner(args, cache_dir="/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/models_cache")

**Initialize trainer**

In [ ]:
trainer = pl.Trainer(**train_params)

**start fine-tuning**

In [ ]:
trainer.fit(model)

In [ ]:
## save the model this way so next time you can load it using T5ForConditionalGeneration.from_pretrained
model.model.save_pretrained('/users/eleves-b/2022/mathias.perez/Desktop/JuridixAssistant/ft_T5/fineTunedT5s/saved_models')

### Eval

For inference we will use the `generate` method with greedy decoding with max length 2.

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

Let's visualize few predictions on test dataset

In [ ]:
path_to_test_csv = ""
dataset = ImdbDataset(tokenizer, path_to_csv=path_to_test_csv,  max_len=512)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
it = iter(loader)

In [ ]:
batch = next(it)
batch["source_ids"].shape

In [ ]:
outs = model.model.generate(input_ids=batch['source_ids'].cuda(),
                              attention_mask=batch['source_mask'].cuda(),
                              max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]

texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
targets = [tokenizer.decode(ids) for ids in batch['target_ids']]

In [ ]:
for i in range(32):
    lines = textwrap.wrap("Initial Query:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual classification and rewriting: %s" % targets[i])
    print("Predicted classification and rewriting: %s" % dec[i])
    print("=====================================================================\n")

Now predict on all the test dataset

In [ ]:
def extract_fields(text):
    """
    Extracts document type and new_query from a formatted string.
    Assumes the text is in the format:
      "type: {doc_type} ; new_query: {new_query}"
      
    Returns:
      A tuple (doc_type, new_query) if both fields are found,
      otherwise (None, None).
    """
    # Use a regex pattern with two capturing groups:
    # Group 1: the document type (any non-semicolon non-space sequence)
    # Group 2: the new_query (any characters until the end)
    pattern = r"^type:\s*([^\s;]+)\s*;\s*new_query:\s*(.+)$"
    match = re.match(pattern, text.lower().strip())
    if match:
        doc_type = match.group(1).strip()
        new_query = match.group(2).strip()
        return doc_type, new_query
    return None, None

In [ ]:
loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.model.eval()
outputs = []
targets = []

for batch in tqdm(loader):
    outs = model.model.generate(
        input_ids=batch['source_ids'].cuda(),
        attention_mask=batch['source_mask'].cuda(),
        max_length=128  # Adjusted max_length to allow for the complete output
    )

    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    batch_targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

    outputs.extend(dec)
    targets.extend(batch_targets)

Let's check if the model generates valid predictions

In [ ]:
# Check for valid formatted outputs.
print("Checking for valid formatted outputs...\n")
for i, (pred_text, tgt_text) in enumerate(zip(outputs, targets)):
    pred_type, pred_new_query = extract_fields(pred_text)
    tgt_type, tgt_new_query = extract_fields(tgt_text)
    
    if pred_type is None or pred_new_query is None:
        print(f"Index {i}: Invalid prediction format: '{pred_text}'")
    if tgt_type is None or tgt_new_query is None:
        print(f"Index {i}: Invalid target format: '{tgt_text}'")

This great is great! Our model hasn't generated any invalid prediction. Let's calculate accuarcy and other metrics

In [ ]:
# Initialize overall counters.
correct_classification = 0
total_samples = len(outputs)

for pred_text, tgt_text in zip(outputs, targets):
    pred_type, _ = extract_fields(pred_text)
    tgt_type, _ = extract_fields(tgt_text)
    
    # Skip if extraction failed.
    if pred_type is None or tgt_type is None:
        continue
    
    if pred_type == tgt_type:
        correct_classification += 1

overall_accuracy = correct_classification / total_samples * 100
print(f"Overall Classification Accuracy: {overall_accuracy:.2f}%")

This cell computes the accuracy for each document category separately.

In [ ]:
from collections import defaultdict

# Initialize per-category counters.
per_category_correct = defaultdict(int)
per_category_total = defaultdict(int)

for pred_text, tgt_text in zip(outputs, targets):
    pred_type, _ = extract_fields(pred_text)
    tgt_type, _ = extract_fields(tgt_text)
    
    # Skip if extraction failed.
    if pred_type is None or tgt_type is None:
        continue
    
    per_category_total[tgt_type] += 1
    if pred_type == tgt_type:
        per_category_correct[tgt_type] += 1

print("Per-Category Accuracy:")
for doc_type in per_category_total:
    cat_accuracy = per_category_correct[doc_type] / per_category_total[doc_type] * 100
    print(f"  {doc_type}: {cat_accuracy:.2f}% ({per_category_correct[doc_type]}/{per_category_total[doc_type]})")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Extract true and predicted document types from the outputs.
true_labels = []
pred_labels = []
for pred_text, tgt_text in zip(outputs, targets):
    pred_type, _ = extract_fields(pred_text)
    tgt_type, _ = extract_fields(tgt_text)
    if pred_type is None or tgt_type is None:
        continue  # Skip entries with invalid formatting.
    true_labels.append(tgt_type)
    pred_labels.append(pred_type)

# Get the sorted list of unique classes.
classes = sorted(list(set(true_labels + pred_labels)))

# Compute the confusion matrix.
cm = confusion_matrix(true_labels, pred_labels, labels=classes)

# Plot the confusion matrix.
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Add text annotations inside each cell.
thresh = cm.max() / 2.0
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()